In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_subjects_electrodestoROIs_dict, load_subjects_electrodestoROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance
import utils
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MOVE ALL FUNCTIONS TO THE TOP!

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [7]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0071', 'D0077', 'D0090', 'D0100', 'D0102', 'D0103']
# subjects = ['D0103'] #testing cuz d0065 being weird

### load subjects electrodes to rois dict

In [10]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = utils.load_subjects_electrodestoROIs_dict(filename)

if subjects_electrodestoROIs_dict is None:
    subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
    utils.make_subjects_electrodestoROIs_dict(subjects)
    subjects_electrodestoROIs_dict = utils.load_subjects_electrodestoROIs_dict(filename)

Loaded data from subjects_electrodestoROIs_dict.json


load accuracy arrays so we can filter by only accurate trials  
combine this code into add_accuracy_to_epochs later!

In [12]:
# this makes numpy arrays for each subject that are 0 or 1 for each trial based on accuracy
from makeRawBehavioralData import main
main()

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path if you're not Jim

acc_array = utils.load_acc_arrays(npy_directory, skip_subjects=['D107'])

In [13]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

In [14]:
# # Define a function to map blockType to congruencyProportion and switchProportion
# def map_block_type(row):
#     if row['blockType'] == 'A':
#         return pd.Series(['25%', '25%'])
#     elif row['blockType'] == 'B':
#         return pd.Series(['25%', '75%'])
#     elif row['blockType'] == 'C':
#         return pd.Series(['75%', '25%'])
#     elif row['blockType'] == 'D':
#         return pd.Series(['75%', '75%'])
#     else:
#         return pd.Series([None, None])

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

### load evoked and stuff for all subjects in a dictionary

In [15]:
# # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # make sure to use the correct column names and values that match with what combinedData uses.

subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# congruency
# output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#     }
# }

# switch type
# output_names = ["Stimulus_r25and75_fixationCrossBase_1sec_mirror", "Stimulus_s25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_r25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#     },
#     "Stimulus_s25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#     }
# }

# #  ir vs is
# output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  cr vs cs
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# #  is vs cs
# output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  ir vs cr
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     }
# }

# # all interaction effects (run this with the anova code. Ugh make everything more modular later.)
output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror", "Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]

output_names_conditions = {
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "r"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "s"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "r"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "s"
    }
}

# # block interaction contrasts for lwpc
# output_names = ["Stimulus_c25_fixationCrossBase_1sec_mirror", "Stimulus_c75_fixationCrossBase_1sec_mirror",  \
#                 "Stimulus_i25_fixationCrossBase_1sec_mirror", "Stimulus_i75_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_c25_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_c75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "congruencyProportion": "25%"
#     },
#     "Stimulus_i25_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "congruencyProportion": "75%"
#     },
#     "Stimulus_i75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "congruencyProportion": "25%"
#     },
# }

# # block interaction contrasts for lwps
# output_names = ["Stimulus_s25_fixationCrossBase_1sec_mirror", "Stimulus_s75_fixationCrossBase_1sec_mirror",  \
#                 "Stimulus_r25_fixationCrossBase_1sec_mirror", "Stimulus_r75_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_s25_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_s75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_r25_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_r75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
# }

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array)

Loading data for subject: D0057
  Loading output: Stimulus_ir_fixationCrossBase_1sec_mirror with conditions: {'congruency': 'i', 'switchType': 'r'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_ir_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
103 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_ir_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\fre

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [16]:
# def get_sig_chans(sub, task, LAB_root=None):
#     """
#     Retrieves the significant channels for a given subject and task from a stored JSON file.
    
#     Parameters:
#         sub (str): Subject ID for which significant channels are retrieved.
#         task (str): The specific task for which data is being processed.
#         LAB_root (str, optional): The root directory where the data is stored. If None, determines the path based on the OS.

#     Returns:
#         dict: A dictionary containing significant channels loaded from the JSON file.
#     """
#     # Determine LAB_root based on the operating system
#     if LAB_root is None:
#         HOME = os.path.expanduser("~")
#         LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

#     # Get data layout
#     layout = get_data(task, root=LAB_root)
#     save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

#     stim_filename = os.path.join(save_dir, f'sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json')
#     stim_sig_chans = load_sig_chans(stim_filename)
#     return stim_sig_chans

# def get_sig_chans_per_subject(subjects, task='GlobalLocal', LAB_root=None):
#     """
#     Retrieves significant channels for a list of subjects for a specified task.
    
#     Parameters:
#         subjects (list of str): List of subject IDs to process.
#         task (str, optional): The specific task for which data is being processed. Defaults to 'GlobalLocal'.
#         LAB_root (str, optional): The root directory where the data is stored. If None, determines the path based on the OS.

#     Returns:
#         dict: A dictionary where keys are subject IDs and values are dictionaries of significant channels for each subject.
#     """
#     # Initialize an empty dictionary to store significant channels per subject
#     sig_chans_per_subject = {}

#     # Populate the dictionary using get_sig_chans for each subject
#     for sub in subjects:
#         sig_chans_per_subject[sub] = get_sig_chans(sub, task, LAB_root)

#     return sig_chans_per_subject


sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [17]:
# def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
#     """
#     Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

#     Args:
#     subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
#     sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
#     roi_list (list): A list of ROIs to filter electrodes.

#     Returns:
#     dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
#     """
#     filtered_electrodes_per_subject = {}

#     for sub, electrodes_dict in subjects_electrodes_dict.items():
#         filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
#                     if any(roi in key for roi in roi_list)}

#         # Aggregate electrodes into a list for each subject
#         filtered_electrodes = []
#         for electrodes in filtered.values():
#             filtered_electrodes.extend(electrodes)

#         filtered_electrodes_per_subject[sub] = filtered_electrodes
#         print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

#     # Now filter for significant electrodes
#     sig_filtered_electrodes_per_subject = {}

#     for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
#         # Retrieve the list of significant channels for the subject
#         sig_chans = sig_chans_per_subject.get(sub, [])

#         # Find the intersection of filtered electrodes and significant channels for the subject
#         sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

#         # Store the significant filtered electrodes for the subject
#         sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
#         print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

#     return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# # Example usage:
# dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
# acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
# parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)

# dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = utils.filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
# # acc_electrodes_per_subject, sig_acc_electrodes_per_subject = utils.filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
# # parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = utils.filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

# sig_electrodes_per_subject_roi = {}
# sig_electrodes_per_subject_roi['dlpfc'] = sig_dlpfc_electrodes_per_subject
# sig_electrodes_per_subject_roi['acc'] = sig_acc_electrodes_per_subject
# sig_electrodes_per_subject_roi['parietal'] = sig_parietal_electrodes_per_subject

In [18]:
# def make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject):
#     """
#     Processes electrodes by ROI and filters significant electrodes.

#     Parameters:
#     - rois_dict: A dictionary mapping each region of interest (ROI) to a list of brain regions.
#     - subjects_electrodestoROIs_dict: A dictionary mapping subjects to their electrode-to-ROI assignments.
#     - sig_chans_per_subject: A dictionary indicating significant channels per subject.

#     Returns:
#     - A tuple of two dictionaries:
#       1. electrodes_per_subject_roi: Electrodes per subject for each ROI.
#       2. sig_electrodes_per_subject_roi: Significant electrodes per subject for each ROI.
#     """
#     electrodes_per_subject_roi = {}
#     sig_electrodes_per_subject_roi = {}

#     for roi_name, roi_regions in rois_dict.items():
#         # Apply the filter_electrodes_by_roi function for each set of ROI regions
#         electrodes_per_subject, sig_electrodes_per_subject = filter_electrodes_by_roi(
#             subjects_electrodestoROIs_dict, sig_chans_per_subject, roi_regions)
        
#         # Store the results in the respective dictionaries
#         electrodes_per_subject_roi[roi_name] = electrodes_per_subject
#         sig_electrodes_per_subject_roi[roi_name] = sig_electrodes_per_subject

#     return electrodes_per_subject_roi, sig_electrodes_per_subject_roi

In [19]:
rois_dict = {
    # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

In [78]:
sig_electrodes_per_subject_roi['lpfc']

{'D0057': ['RAI6', 'RPI14'],
 'D0059': ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16'],
 'D0063': ['LMSF5',
  'LPSF12',
  'RAI4',
  'RAI6',
  'RAMF12',
  'RMMF13',
  'RMMF14',
  'RMMF10'],
 'D0065': ['RASF14'],
 'D0069': [],
 'D0071': ['RFO14',
  'RIA4',
  'RIP6',
  'RIA5',
  'RIA11',
  'RIA12',
  'RIA14',
  'RIA16'],
 'D0077': [],
 'D0090': ['RIA6', 'RIA12'],
 'D0094': ['LFAM8',
  'LFAM9',
  'LFAM10',
  'LFPM10',
  'LFPM12',
  'LFAI5',
  'LPAI9',
  'LFAI9',
  'LFAI10',
  'LIA11'],
 'D0100': [],
 'D0102': ['RFO13', 'RFO14', 'RFAM15'],
 'D0103': ['LFAM8', 'LFAM9', 'LAI13', 'LAI14', 'LAI4', 'LFAI4']}

get total number of electrodes (make this modular with roi later once everything works)

In [20]:
# def calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi):
#     """
#     Calculates the total number of significant and total electrodes for each ROI across all subjects.

#     Parameters:
#     - sig_electrodes_per_subject_roi: A dictionary containing significant electrodes per subject for each ROI.
#     - electrodes_per_subject_roi: A dictionary containing all electrodes per subject for each ROI.

#     Returns:
#     - A dictionary containing the counts of significant and total electrodes for each ROI.
#     """
#     total_electrodes_info = {}

#     for roi in sig_electrodes_per_subject_roi:
#         # Calculate total significant electrodes for the current ROI
#         total_sig_entries = sum(len(sig_electrodes_per_subject_roi[roi][sub]) for sub in sig_electrodes_per_subject_roi[roi])
#         # Calculate total electrodes for the current ROI
#         total_entries = sum(len(electrodes_per_subject_roi[roi][sub]) for sub in electrodes_per_subject_roi[roi])

#         # Store the results in the dictionary
#         total_electrodes_info[roi] = {
#             'total_significant_electrodes': total_sig_entries,
#             'total_electrodes': total_entries
#         }

#     return total_electrodes_info

In [21]:
# Example usage:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 105


### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is 1/23 old approach of avg across trials first. Time perm cluster stats.

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)  
this code is so bad right now, turn into a function later  

trialAvg is for the time perm cluster stats  
timeAvg_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond is for the window stats (not sure if this is even right)  



4/30 try to make time perm stats more modular, and reusable  
also remember that time perm cluster stats only compares two output names. Currently it just grabs the first two.

In [95]:
def initialize_output_data(rois, output_names):
    """
    Initialize dictionaries for storing data across different outputs and ROIs.
    """
    return {output_name: {roi: [] for roi in rois} for output_name in output_names}

# Correctly call and use the functions
def process_data_for_roi(subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices):
    """
    Process data by ROI, calculating averages for different time windows for either the first two outputs or all outputs, depending on the analysis purpose.
    """

    # Initialize data structures for trial averages, trial standard deviations, and time averages
    data_trialAvg_lists = initialize_data_structures(rois, output_names)
    data_trialStd_lists = initialize_data_structures(rois, output_names)
    data_timeAvg_lists = {suffix: initialize_data_structures(rois, output_names) for suffix in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']}
    overall_electrode_mapping = []
    electrode_mapping_per_roi = {roi: [] for roi in rois}  # Reinitialize for each processing run

    for sub in subjects:
        for roi in rois:
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            print(f"Subject: {sub}, ROI: {roi}, Num of Sig Electrodes: {len(sig_electrodes)}")  # Debug print

            if not sig_electrodes:
                continue

            for output_name in output_names:
                epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)

                # Append mapping information for use in ANOVA.
                for electrode in sig_electrodes:
                    index = len(overall_electrode_mapping)
                    overall_electrode_mapping.append((sub, roi, electrode, index))
                    index_roi = len(electrode_mapping_per_roi[roi])
                    electrode_mapping_per_roi[roi].append((sub, electrode, index_roi))

                # Compute trial averages and standard deviations once per output per subject per ROI
                trial_avg, trial_std, _ = filter_and_average_epochs(epochs, start_idx=None, end_idx=None)
                data_trialAvg_lists[output_name][roi].append(trial_avg)
                data_trialStd_lists[output_name][roi].append(trial_std)

                # compute time average for each output per subject per roi for each time window. But why don't we look at standard deviation? 4/30
                for suffix, (start_idx, end_idx) in time_indices.items():
                    _, _, time_avg = filter_and_average_epochs(epochs, start_idx, end_idx)
                    data_timeAvg_lists[suffix][output_name][roi].append(time_avg)

    return data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi

def concatenate_data(data_lists, rois, output_names):
    """
    Concatenate data across subjects for each ROI and condition.
    """
    concatenated_data = {output_name: {roi: np.concatenate(data_lists[output_name][roi], axis=0) for roi in rois} for output_name in output_names}
    return concatenated_data

def calculate_mean_and_sem(concatenated_data, rois, output_names):
    """
    Calculate mean and SEM across electrodes for all time windows and rois
    """
    mean_and_sem = {roi: {output_name: {} for output_name in output_names} for roi in rois}
    for roi in rois:
        for output_name in output_names:
            trial_data = concatenated_data[output_name][roi]
            mean = np.nanmean(trial_data, axis=0)
            sem = np.std(trial_data, axis=0, ddof=1) / np.sqrt(trial_data.shape[0])
            mean_and_sem[roi][output_name] = {'mean': mean, 'sem': sem}
    return mean_and_sem

def calculate_time_perm_cluster_for_each_roi(concatenated_data, rois, output_names, alpha=0.05, n_jobs=6):
    """
    Perform time permutation cluster tests between the first two outputs for each ROI.
    Assumes that there are at least two output conditions to compare.
    """
    time_perm_cluster_results = {}
    for roi in rois:
        time_perm_cluster_results[roi] = time_perm_cluster(
            concatenated_data[output_names[0]][roi],
            concatenated_data[output_names[1]][roi], alpha, n_jobs=n_jobs
        )
    return time_perm_cluster_results

In [107]:
# Define time indices as in your original code
time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096),
    'fullSecond': (2048, 4096)
}

# Select output names based on whether the processing is for a permutation test (first two outputs) or ANOVA (all outputs).
for_perm_test = True
relevant_output_names = output_names[:2] if for_perm_test else output_names

# Process the data
data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi = process_data_for_roi(
    subjects_mne_objects, relevant_output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

# Concatenate the data
concatenated_trialAvg_data = concatenate_data(data_trialAvg_lists, rois, relevant_output_names)

# Calculate means and sems
mean_and_sem = calculate_mean_and_sem(concatenated_trialAvg_data, rois, relevant_output_names)

# Perform statistical tests
time_perm_cluster_results = calculate_time_perm_cluster_for_each_roi(concatenated_trialAvg_data, rois, relevant_output_names)

Subject: D0057, ROI: lpfc, Num of Sig Electrodes: 2
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:130: RuntimeWarning: Mean of empty slice
  time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0059, ROI: lpfc, Num of Sig Electrodes: 4
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0063, ROI: lpfc, Num of Sig Electrodes: 8
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0065, ROI: lpfc, Num of Sig Electrodes: 1
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0069, ROI: lpfc, Num of Sig Electrodes: 0
Subject: D0071, ROI: lpfc, Num of Sig Electrodes: 8
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0077, ROI:

full pipeline analysis for time perm cluster stats, untested 5/4. Replaces the above cell.

In [ ]:
def run_time_perm_cluster_stats(subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, for_perm_test=True):
    """
    Execute the full time perm cluster stats pipeline, from data processing to statistical testing.

    Parameters:
    subjects_mne_objects: Dictionary with MNE data objects for each subject.
    output_names: List of output condition names.
    rois: List of regions of interest.
    subjects: List of subject identifiers.
    sig_electrodes_per_subject_roi: Mapping of significant electrodes per subject per ROI.
    for_perm_test: Boolean flag to determine which outputs to process for testing.

    Returns:
    A dictionary with trial averages, trial standard deviations, time averages, and time perm cluster results.
    """
    print("Starting the analysis... Let's crunch some numbers!")
    
    # Define time indices. Should probably replace these magic numbers with sampling rate calc...5/4
    time_indices = {
        'firstHalfSecond': (2048, 3072),
        'secondHalfSecond': (3072, 4096),
        'fullSecond': (2048, 4096)
    }

    # Select output names based on whether the processing is for a permutation test (first two outputs) or ANOVA (all outputs).
    relevant_output_names = output_names[:2] if for_perm_test else output_names

    # Process the data
    data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi = process_data_for_roi(
        subjects_mne_objects, relevant_output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

    print("Data processing complete. Now let's concatenate the results and see what we've got!")

    # Concatenate the data
    concatenated_trialAvg_data = concatenate_data(data_trialAvg_lists, rois, relevant_output_names)

    # Calculate means and sems
    mean_and_sem = calculate_mean_and_sem(concatenated_trialAvg_data, rois, relevant_output_names)

    print("Mean and SEM calculation done. It's time for some statistical action!")

    # Perform statistical tests
    if for_perm_test:
        print("Running permutation tests...")
        time_perm_cluster_results = calculate_time_perm_cluster_for_each_roi(concatenated_trialAvg_data, rois, relevant_output_names)
    else:
        print("No permutation test specified. Moving on...")
        time_perm_cluster_results = None

    print("Analysis complete! Returning all the juicy details now.")

    return {
        'Trial Averages': data_trialAvg_lists,
        'Trial Standard Deviations': data_trialStd_lists,
        'Time Averages': data_timeAvg_lists,
        'Overall Electrode Mapping': overall_electrode_mapping,
        'Electrode Mapping per ROI': electrode_mapping_per_roi,
        'Mean and SEM': mean_and_sem,
        'Permutation Test Results': time_perm_cluster_results
    }

# Usage:
time_perm_cluster_results = run_time_perm_cluster_stats(subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, for_perm_test=True)


### do window stats  

shuffle test (perm test). This basically time perm cluster but avg across time.

In [ ]:
# Select output names based on whether the processing is for a permutation test (first two outputs) or ANOVA (all outputs).
for_perm_test = True
relevant_output_names = output_names[:2] if for_perm_test else output_names

# Process the data
data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi = process_data_for_roi(
    subjects_mne_objects, relevant_output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

this code is old as of 5/2

In [ ]:
# # Assuming the functions perform_permutation_test_within_electrodes and perform_permutation_test_across_electrodes return lists of p-values (and get loaded in properly)
# p_values = {}
# for roi in rois:
#     # Initialize p_values[roi] as a dictionary. Initialize dicts for all time windows.
#     p_values[roi] = {}
#     p_values[roi]['firstHalfSecond'] = {}
#     p_values[roi]['secondHalfSecond'] = {}
#     p_values[roi]['fullSecond'] = {}

#     # Perform the tests and store results
#     p_values[roi]['firstHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)
#     p_values[roi]['firstHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)

#     p_values[roi]['secondHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_secondHalfSecond_list[roi], output_1_data_timeAvg_secondHalfSecond_list[roi], n_permutations=10000)
#     p_values[roi]['secondHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_secondHalfSecond_list[roi], output_1_data_timeAvg_secondHalfSecond_list[roi], n_permutations=10000)

#     p_values[roi]['fullSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_fullSecond_list[roi], output_1_data_timeAvg_fullSecond_list[roi], n_permutations=10000)
#     p_values[roi]['fullSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_fullSecond_list[roi], output_1_data_timeAvg_fullSecond_list[roi], n_permutations=10000)


# # I'm pretty sure we should only do within electrode and not across electrodes for the all_p_values list. So don't iterate over test_types here. 3/11.
# all_p_values = {}
# all_p_values['firstHalfSecond'] = []
# all_p_values['secondHalfSecond'] = []
# all_p_values['fullSecond'] = []

# for roi in p_values:
#     for test_type in p_values[roi]['firstHalfSecond']:
#         p = p_values[roi]['firstHalfSecond'][test_type]
#         if isinstance(p, list):
#             all_p_values['firstHalfSecond'].extend(p)
#         else:  # Assume it's a single float value
#             all_p_values['firstHalfSecond'].append(p)

#     for test_type in p_values[roi]['secondHalfSecond']:
#         p = p_values[roi]['secondHalfSecond'][test_type]
#         if isinstance(p, list):
#             all_p_values['secondHalfSecond'].extend(p)
#         else:  # Assume it's a single float value
#             all_p_values['secondHalfSecond'].append(p)

#     for test_type in p_values[roi]['fullSecond']:
#         p = p_values[roi]['fullSecond'][test_type]
#         if isinstance(p, list):
#             all_p_values['fullSecond'].extend(p)
#         else:  # Assume it's a single float value
#             all_p_values['fullSecond'].append(p)

# # Apply FDR correction
# _, adjusted_p_values_firstHalfSecond = multipletests(all_p_values['firstHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
# _, adjusted_p_values_secondHalfSecond = multipletestsload(all_p_values['secondHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
# _, adjusted_p_values_fullSecond = multipletests(all_p_values['fullSecond'], alpha=0.05, method='fdr_bh')[:2]

# # Incorporating adjusted p-values back into the structure is a bit more complex and depends on how you want to use them next

functions

In [119]:
def perform_window_perm_tests(data_timeAvg_lists, relevant_output_names, rois, n_permutations=10000):
    p_values = {}
    for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
        p_values[time_window] = {}
        for roi in rois:
            p_values[time_window][roi] = {}
            output_data_lists = data_timeAvg_lists[time_window]
            if relevant_output_names[0] in output_data_lists and relevant_output_names[1] in output_data_lists:
                p_values[time_window][roi]['within'] = perform_permutation_test_within_electrodes(
                    output_data_lists[relevant_output_names[0]][roi], 
                    output_data_lists[relevant_output_names[1]][roi], 
                    n_permutations=n_permutations
                )
                p_values[time_window][roi]['across'] = perform_permutation_test_across_electrodes(
                    output_data_lists[relevant_output_names[0]][roi], 
                    output_data_lists[relevant_output_names[1]][roi], 
                    n_permutations=n_permutations
                )
    return p_values

def apply_fdr_correction_window_perm_test(p_values):
    all_p_values = {time_window: [] for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']}
    index_map = {time_window: [] for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']}

    for time_window, roi_data in p_values.items():
        for roi, tests in roi_data.items():
            for test_type, p in tests.items():
                p_list = p if isinstance(p, list) else [p]
                all_p_values[time_window].extend(p_list)
                index_map[time_window].extend([(roi, test_type)] * len(p_list))

    adjusted_p_values = {}
    for time_window, p_values_list in all_p_values.items():
        _, corrected_p_values = multipletests(p_values_list, alpha=0.05, method='fdr_bh')[:2]
        adjusted_p_values[time_window] = corrected_p_values

    return adjusted_p_values, index_map

def integrate_adjusted_p_values(p_values, adjusted_p_values, index_map):
    for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
        for i, adjusted_p in enumerate(adjusted_p_values[time_window]):
            roi, test_type = index_map[time_window][i]
            if 'adjusted' not in p_values[time_window][roi]:
                p_values[time_window][roi]['adjusted'] = {}
            if test_type not in p_values[time_window][roi]['adjusted']:
                p_values[time_window][roi]['adjusted'][test_type] = []
            p_values[time_window][roi]['adjusted'][test_type].append(adjusted_p)


In [121]:
# Step 1: Perform the permutation tests
p_values = perform_window_perm_tests(data_timeAvg_lists, relevant_output_names, rois)

# Step 2: Apply FDR correction and get the index map
adjusted_p_values, index_map = apply_fdr_correction_window_perm_test(p_values)

# Step 3: Reintegrate the adjusted p-values
integrate_adjusted_p_values(p_values, adjusted_p_values, index_map)


Subject 0 - Condition 0 shape: (103, 2), Condition 1 shape: (118, 2)
Subject 1 - Condition 0 shape: (114, 4), Condition 1 shape: (107, 4)
Subject 2 - Condition 0 shape: (106, 8), Condition 1 shape: (116, 8)
Subject 3 - Condition 0 shape: (119, 1), Condition 1 shape: (103, 1)
Subject 4 - Condition 0 shape: (115, 8), Condition 1 shape: (109, 8)
Subject 5 - Condition 0 shape: (113, 2), Condition 1 shape: (109, 2)
Subject 6 - Condition 0 shape: (112, 10), Condition 1 shape: (108, 10)
Subject 7 - Condition 0 shape: (107, 3), Condition 1 shape: (114, 3)
Subject 8 - Condition 0 shape: (114, 6), Condition 1 shape: (107, 6)
Subject 0 - Condition 0 shape: (103, 2), Condition 1 shape: (118, 2)
Subject 1 - Condition 0 shape: (114, 4), Condition 1 shape: (107, 4)
Subject 2 - Condition 0 shape: (106, 8), Condition 1 shape: (116, 8)
Subject 3 - Condition 0 shape: (119, 1), Condition 1 shape: (103, 1)
Subject 4 - Condition 0 shape: (115, 8), Condition 1 shape: (109, 8)
Subject 5 - Condition 0 shape: (

In [122]:
p_values['firstHalfSecond']['lpfc']

{'within': [0.2619,
  0.6322,
  0.3152,
  0.4441,
  0.1199,
  0.9227,
  0.8936,
  0.0346,
  0.5904,
  0.1598,
  0.3105,
  0.9758,
  0.4072,
  0.1786,
  0.5858,
  0.0,
  0.3179,
  0.3552,
  0.0005,
  0.0037,
  0.044,
  0.0203,
  0.1662,
  0.4445,
  0.9832,
  0.7989,
  0.3331,
  0.12,
  0.0887,
  0.1274,
  0.8001,
  0.4971,
  0.2895,
  0.1721,
  0.7462,
  0.2492,
  0.8879,
  0.1852,
  0.7772,
  0.7768,
  0.8209,
  0.753,
  0.565,
  0.9631],
 'across': 0.4728,
 'adjusted': {'within': [0.6812142857142858,
   0.8890312499999999,
   0.6812142857142858,
   0.7693269230769232,
   0.5556000000000001,
   0.9832,
   0.980780487804878,
   0.3114,
   0.8570322580645162,
   0.5556000000000001,
   0.6812142857142858,
   0.9832,
   0.7635000000000001,
   0.5556000000000001,
   0.8570322580645162,
   0.0,
   0.6812142857142858,
   0.6949565217391305,
   0.01125,
   0.0555,
   0.32999999999999996,
   0.22837499999999997,
   0.5556000000000001,
   0.7693269230769232,
   0.9832,
   0.9471923076923077,
   

### do 2x2 anova for interaction effects 
this requires reloading in all four conditions (four this time cuz interaction contrasts).  
ONLY RUN THIS WHEN LOADING IN THE FOUR INTERACTION CONTRASTS RIGHT NOW.  
Integrate with other stats and plotting and stuff later.

In [ ]:
# Time windows
# define these based on epoch start and also sampling rate
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096
# Function to create a nested dictionary for each output_name with ROIs as keys
def initialize_output_data(rois, output_names):
    return {output_name: {roi: [] for roi in rois} for output_name in output_names}

# Assuming output_names contains all four conditions
# Initializing dictionaries
output_data_trialAvg_lists = initialize_output_data(rois, output_names)
output_data_trialStd_lists = initialize_output_data(rois, output_names)
output_data_timeAvg_firstHalfSecond_lists = initialize_output_data(rois, output_names)
output_data_timeAvg_secondHalfSecond_lists = initialize_output_data(rois, output_names)
output_data_timeAvg_fullSecond_lists = initialize_output_data(rois, output_names)

# Initialize a dictionary to hold mappings
overall_electrode_mapping = []

# Initialize a dictionary to hold mappings for each ROI
electrode_mapping_per_roi = {roi: [] for roi in rois}
for sub in subjects:
    for roi in rois:
        for output_name in output_names:
            # Determine significant electrodes for the current ROI and subject
            # sig_electrodes = sig_dlpfc_electrodes_per_subject.get(sub, []) if roi == 'dlpfc' else sig_acc_electrodes_per_subject.get(sub, []) if roi == 'acc' else sig_parietal_electrodes_per_subject.get(sub, [])
            
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            print('sub', sub)
            print('sig elecs:', sig_electrodes)
            
            if not sig_electrodes:  # Skip if no significant electrodes
                continue
                        
            for electrode in sig_electrodes:
                # For each significant electrode, append a tuple to the mapping list
                # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
                # The index can be the current length of the list before appending
                index = len(overall_electrode_mapping)
                overall_electrode_mapping.append((sub, roi, electrode, output_name, index))  

                # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
                # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
                index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
                electrode_mapping_per_roi[roi].append((sub, electrode, output_name, index))
                
            # Load trial-level data for the current condition and pick significant electrodes
            epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
            # print(epochs.get_data().shape)
            # Calculate averages for each time window
            trial_avg, trial_std, time_avg_firstHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
            _, _, time_avg_secondHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
            _, _, time_avg_fullSecond = utils.filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)
            print('time avg full second shape:', time_avg_fullSecond.shape)

            # Append the results to their respective lists
            output_data_trialAvg_lists[output_name][roi].append(trial_avg)
            output_data_trialStd_lists[output_name][roi].append(trial_std)
            output_data_timeAvg_firstHalfSecond_lists[output_name][roi].append(time_avg_firstHalfSecond)
            output_data_timeAvg_secondHalfSecond_lists[output_name][roi].append(time_avg_secondHalfSecond)
            output_data_timeAvg_fullSecond_lists[output_name][roi].append(time_avg_fullSecond)


# After collecting all data, concatenate across subjects for each roi and condition
concatenated_trialAvg_data = {}
concatenated_trialStd_data = {}

for roi in rois:
    concatenated_trialAvg_data[roi] = {}
    concatenated_trialStd_data[roi] = {}

    for output_name in output_names:
        concatenated_trialAvg_data[roi][output_name] = np.concatenate(output_data_trialAvg_lists[output_name][roi], axis=0)
        concatenated_trialStd_data[roi][output_name] = np.concatenate(output_data_trialStd_lists[output_name][roi], axis=0)


# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}
for roi in rois:
    overall_averages[roi] = {}
    overall_sems[roi] = {}
    for output_name in output_names:
        trialAvg_data = concatenated_trialAvg_data[roi][output_name]
        overall_averages[roi][output_name] = np.nanmean(trialAvg_data, axis=0)
        overall_sems[roi][output_name] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])

the subject indexing is being thrown off by the subjects that DONT have electrodes in this roi. Maybe skip those subjects for the index? Or make subject index consistent across variables.. 4/1

get subjects that have data so that indexing isn't thrown off

In [ ]:
sig_electrodes_per_subject_roi['lpfc']

In [ ]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Example structure for organizing data for ANOVA with four conditions
data_for_anova = []


# Function to process and append data for ANOVA from time-averaged lists
# Adapted function to include Congruency and SwitchType
# modifying this to include time windows as another factor 4/4! Use code before 4/4 if don't want to include time windows.
def process_and_append_data_for_anova(time_averaged_lists_dict):
    for time_window, lists in time_averaged_lists_dict.items():
        for output_name in output_names:
            print('output name:', output_name)
            # Dynamically get condition types and their values for the current output_name
            conditions = output_names_conditions[output_name]
            
            for roi in rois: #this is good cuz it loops through rois 3/6, the trial level one should copy this logic
                sig_electrodes_per_subject = sig_electrodes_per_subject_roi[roi]
                subjects_with_data = [subject for subject, electrodes in sig_electrodes_per_subject.items() if electrodes] # add this line to skip over subjects without data 4/1
                for subject_index, subject_data in enumerate(lists[output_name][roi]):
                    subject_id = subjects_with_data[subject_index]

                    # Skip this subject if there are no significant electrodes for them in this ROI
                    if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                        continue

                    # Calculate the mean across trials for each electrode
                    mean_activity_per_electrode = np.nanmean(subject_data, axis=0)
                    # untested making this more modular 2/27
                    for electrode_index, mean_activity in enumerate(mean_activity_per_electrode):
                        print('electrode index:', electrode_index)
                        electrode_name = sig_electrodes_per_subject[subject_id][electrode_index]
                        print(electrode_name)
                        # Prepare data dictionary, starting with fixed attributes
                        data_dict = {
                            'SubjectID': subject_id,
                            'Electrode': electrode_name,
                            'ROI': roi,
                            'TimeWindow': time_window,
                            'MeanActivity': mean_activity
                        }

                        # Dynamically add condition types and their values
                        data_dict.update(conditions)

                        # Append the organized data to the list
                        data_for_anova.append(data_dict)

# Create a time averaged lists dictionary to pass in to the process and append data for anova function
                    
# use this one to compare early vs late vs all time
# time_averaged_lists = {
#         "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
#         "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists,
#         "FullSecond": output_data_timeAvg_fullSecond_lists
# }
                    
# use this one to just compare early and late time
time_averaged_lists_dict = {
        "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
        "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists
}

process_and_append_data_for_anova(time_averaged_lists_dict)
# Convert to DataFrame
df_for_anova = pd.DataFrame(data_for_anova)

In [ ]:
df_for_anova

now actually run anova

In [ ]:
def perform_modular_anova_all_time_windows(df, output_names_conditions, save_dir, save_name_prefix):
    # Dynamically construct the model formula based on condition keys and include TimeWindow
    condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
    formula_terms = ' + '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
    interaction_terms = ' * '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
    formula = f'MeanActivity ~ {formula_terms} + {interaction_terms}'

    # Define the model
    model = ols(formula, data=df).fit()

    # Perform the ANOVA
    anova_results = anova_lm(model, typ=2)

    # Define the base part of the results file name
    results_file_path = os.path.join(save_dir, f"{save_name_prefix}_ANOVAacrossElectrodes_allTimeWindows.txt")

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(anova_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"ANOVA results for all time windows saved to: {results_file_path}")

    # Print the results
    print(anova_results)

    return anova_results


In [ ]:
# Join all the ROIs with '_' as the separator
rois_suffix = '_'.join(rois)

congruency vs congruency proportion

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova_all_time_windows(df_for_anova, output_names_conditions, save_dir, f'congruency_congruencyProportion_{rois_suffix}')

do switch type vs switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova_all_time_windows(df_for_anova, output_names_conditions, save_dir, f'switchType_switchProportion_{rois_suffix}')

do congruency vs switch type

test the new across time windows anova 4/4

In [ ]:
if 'Stimulus_cs_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova_all_time_windows(df_for_anova, output_names_conditions, save_dir, f'congruency_switchType_{rois_suffix}')

okay now do within-electrode anova too

In [ ]:
data_for_anova = []

def process_and_append_trial_data_for_anova(time_averaged_lists, output_names_conditions):
    for time_window, lists in time_averaged_lists.items():
        for output_name, conditions in output_names_conditions.items():
            for roi in rois:
                sig_electrodes_per_subject = sig_electrodes_per_subject_roi[roi]
                subjects_with_data = [subject for subject, electrodes in sig_electrodes_per_subject.items() if electrodes] # Skip over subjects without data
                for subject_index, subject_data in enumerate(lists[output_name][roi]):
                    subject_id = subjects_with_data[subject_index]

                    if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                        continue

                    for trial_index, trial_data in enumerate(subject_data):
                        # Skip trials with any missing data or incorrect length
                        if np.any(np.isnan(trial_data)) or len(trial_data) != len(sig_electrodes_per_subject[subject_id]):
                            continue

                        for electrode_index, electrode_name in enumerate(sig_electrodes_per_subject[subject_id]):
                            activity = trial_data[electrode_index] if electrode_index < len(trial_data) else np.nan

                            # Prepare the data dictionary
                            data_dict = {
                                'SubjectID': subject_id,
                                'Electrode': electrode_name,
                                'ROI': roi,
                                'TimeWindow': time_window,
                                'Trial': trial_index + 1,
                                'Activity': activity
                            }

                            # Dynamically add condition types and their values
                            data_dict.update(conditions)

                            data_for_anova.append(data_dict)

# Example usage with the `time_averaged_lists` dictionary
time_averaged_lists = {
    "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
    "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists,
    # "FullSecond": output_data_timeAvg_fullSecond_lists  # Uncomment or comment based on your needs
}

process_and_append_trial_data_for_anova(time_averaged_lists, output_names_conditions)

# Convert to DataFrame
df_for_trial_level_anova = pd.DataFrame(data_for_anova)


In [ ]:
# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova_roi(df, output_names_conditions, save_dir, save_name):
    '''
    This gets if an electrode is significant for specific time windows. It does not get their interaction.
    '''
    import json
    results = []
    significant_effects_structure = {}

    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique(): #this is wrong cuz then it only does dlpfc, fix this 4/1
            for time_window in df['TimeWindow'].unique():
                df_filtered = df[(df['SubjectID'] == subject_id) & 
                                 (df['Electrode'] == electrode) & 
                                 (df['TimeWindow'] == time_window)]
                
                if df_filtered.empty:
                    continue
                
                # Dynamically construct the formula based on condition keys present in the DataFrame
                condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
                formula_terms = ' + '.join([f'C({key})' for key in condition_keys])
                interaction_terms = ' * '.join([f'C({key})' for key in condition_keys])
                formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                # Perform the ANOVA
                model = ols(formula, data=df_filtered).fit()
                anova_results = anova_lm(model, typ=2)
                
                # Append the results
                results.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode,
                    'TimeWindow': time_window,
                    'ANOVA_Results': anova_results
                })
    
    # Join all the ROIs with '_' as the separator
    rois_suffix = '_'.join(rois)

    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes_{rois_suffix}.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes_{rois_suffix}.txt"
    significantEffectsStructureFilename = f"{save_name}_significantEffectsStructure_{rois_suffix}.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        time_window = result['TimeWindow']
        
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}, Time Window: {time_window}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'TimeWindow': time_window,
                'SignificantEffects': significant_effects
            })

        # Extract significant effects for the current result. Basically just get the p-value. 3/19.
        sig_effects_just_p_values = utils.extract_significant_effects(anova_table)
        
        if sig_effects_just_p_values:
            # Ensure subject_id and electrode keys exist
            if subject_id not in significant_effects_structure:
                significant_effects_structure[subject_id] = {}
            if electrode not in significant_effects_structure[subject_id]:
                significant_effects_structure[subject_id][electrode] = {}
            
            # Assign the significant effects and their p-values to the correct structure
            significant_effects_structure[subject_id][electrode][time_window] = sig_effects_just_p_values    

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    significant_effects_structure_file_path = os.path.join(save_dir, significantEffectsStructureFilename)
    # Save the ANOVA results to a json file (if this works, change the others to json files too)
    with open(significant_effects_structure_file_path, 'w') as file:
        json.dump(significant_effects_structure, file, indent=4)

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_effects_structure saved to: {significant_effects_structure_file_path}")

    return results, significant_results, significant_effects_structure

In [ ]:
# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova_roi_timeWindowInteractions(df, output_names_conditions, save_dir, save_name):
    '''
    This gets if the main and interaction effect of time window is significant for an electrode. AKA is overall or condition-specific activity different across differnet time windows?
    It does not tell you which time windows are significant. 
    '''
    import json
    results = []
    significant_effects_structure = {}

    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique(): #this is wrong cuz then it only does dlpfc, fix this 4/1
                df_filtered = df[(df['SubjectID'] == subject_id) & 
                                 (df['Electrode'] == electrode)]
                
                if df_filtered.empty:
                    continue
                
                # Dynamically construct the formula based on condition keys present in the DataFrame
                condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
                formula_terms = ' + '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
                interaction_terms = ' * '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
                formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                # Perform the ANOVA
                model = ols(formula, data=df_filtered).fit()
                anova_results = anova_lm(model, typ=2)
                
                # Append the results
                results.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode,
                    'ANOVA_Results': anova_results
                })
    
    # Join all the ROIs with '_' as the separator
    rois_suffix = '_'.join(rois)

    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes_{rois_suffix}.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes_{rois_suffix}.txt"
    significantEffectsStructureFilename = f"{save_name}_significantEffectsStructure_{rois_suffix}.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'SignificantEffects': significant_effects
            })

        # Extract significant effects for the current result. Basically just get the p-value. 3/19.
        sig_effects_just_p_values = utils.extract_significant_effects(anova_table)
        
        if sig_effects_just_p_values:
            # Ensure subject_id and electrode keys exist
            if subject_id not in significant_effects_structure:
                significant_effects_structure[subject_id] = {}
            if electrode not in significant_effects_structure[subject_id]:
                significant_effects_structure[subject_id][electrode] = {}
            
            # Assign the significant effects and their p-values to the correct structure
            significant_effects_structure[subject_id][electrode] = sig_effects_just_p_values    

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    significant_effects_structure_file_path = os.path.join(save_dir, significantEffectsStructureFilename)
    # Save the ANOVA results to a json file (if this works, change the others to json files too)
    with open(significant_effects_structure_file_path, 'w') as file:
        json.dump(significant_effects_structure, file, indent=4)

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_effects_structure saved to: {significant_effects_structure_file_path}")

    return results, significant_results, significant_effects_structure


congruency as function of congruency proportion  
maybe make the save_name based on the conditions..?

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_roi(df_for_trial_level_anova, output_names_conditions, save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    results_timeWindowInteraction, significant_results_timeWindowInteraction, significant_effects_structure_timeWindowInteraction = perform_modular_within_electrode_anova_roi_timeWindowInteractions(df_for_trial_level_anova, output_names_conditions, save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes_timeWindowInteractions')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

switch type as function of switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results. significant_effects_structure = perform_modular_within_electrode_anova_roi(df_for_trial_level_anova, output_names_conditions, save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")  

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    results_timeWindowInteraction, significant_results_timeWindowInteraction, significant_effects_structure_timeWindowInteraction = perform_modular_within_electrode_anova_roi_timeWindowInteractions(df_for_trial_level_anova, output_names_conditions, save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes_timeWindowInteractions')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")

congruency as function of switch type

4/4 do both perform_modular_within_electrode_anova_roi and perform_modular_within_electrode_anova_roi_timeWindowInteractions. This will tell us which time windows have significant activity in an electrode AND if there is significant differences in activity across time windows in an electrode.

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_roi(df_for_trial_level_anova, output_names_conditions, save_dir, 'congruency_switchType_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    results_timeWindowInteraction, significant_results_timeWindowInteraction, significant_effects_structure_timeWindowInteraction = perform_modular_within_electrode_anova_roi_timeWindowInteractions(df_for_trial_level_anova, output_names_conditions, save_dir, 'congruency_switchType_ANOVAwithinElectrodes_timeWindowInteractions')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
results

In [ ]:
significant_results 

### plot and QC stats

plot time perm cluster stats (don't run this immediately below cell if didn't do time perm cluster)

In [ ]:
# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(time_perm_cluster_results['dlpfc'])
# plt.xlabel('Timepoints')
# plt.ylabel('Significance (0 or 1)')
# plt.title('Permutation Test Significance Over Time')
# plt.show()

### plot interaction effects (only do this when load in all four of them)

https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
# # add the other conditions and give them condition names and colors too
plotting_parameters = {
    'Stimulus_r25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'repeat',
        'color': 'red',
        "line_style": "-"
    },
    'Stimulus_s25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'switch',
        'color': 'green',
        "line_style": "-"
    },
    'Stimulus_c25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'congruent',
        'color': 'blue',
        "line_style": "-"
    },
    'Stimulus_i25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'incongruent',
        'color': 'orange',
        "line_style": "-"
    },
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "condition_name": "IR",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "condition_name": "IS",
        "color": "blue",
        "line_style": "--"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "condition_name": "CR",
        "color": "red",
        "line_style": "-"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "condition_name": "CS",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c25_fixationCrossBase_1sec_mirror": {
        "condition_name": "c25",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c75_fixationCrossBase_1sec_mirror": {
        "condition_name": "c75",
        "color": "red",
        "line_style": "-"
    },
    "Stimulus_i25_fixationCrossBase_1sec_mirror": {
        "condition_name": "i25",
        "color": "blue",
        "line_style": "--"
    },
    "Stimulus_i75_fixationCrossBase_1sec_mirror": {
        "condition_name": "i75",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_s25_fixationCrossBase_1sec_mirror": {
        "condition_name": "s25",
        "color": "green",
        "line_style": "--"
    },
    "Stimulus_s75_fixationCrossBase_1sec_mirror": {
        "condition_name": "s75",
        "color": "green",
        "line_style": "-"
    },
    "Stimulus_r25_fixationCrossBase_1sec_mirror": {
        "condition_name": "r25",
        "color": "pink",
        "line_style": "--"
    },
    "Stimulus_r75_fixationCrossBase_1sec_mirror": {
        "condition_name": "r75",
        "color": "pink",
        "line_style": "-"
    },

}

# # Save the dictionary to a file
# with open('plotting_parameters.json', 'w') as file:
#     json.dump(plotting_parameters, file, indent=4)

In [ ]:
utils.make_plotting_parameters() #make plotting parameters. Modify colors and line types in misc_functions.

# Load the dictionary from the file
with open('plotting_parameters.json', 'r') as file:
    plotting_parameters = json.load(file)

print(plotting_parameters)

In [ ]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

def plot_interact_effects_modular_roi(roi, save_dir, save_name, overall_averages, overall_sems, output_names, plotting_parameters):
    # Base setup for directories and file paths
    save_path = os.path.join(save_dir, f'avg_{roi}_{save_name}_interactEffects_zscore_roi.png')

    # Initialize plot
    plt.figure(figsize=(10, 6))

    # Dynamically select the first subject and use it to extract times
    first_subject_id = next(iter(subjects_mne_objects))
    example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
    times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

    overall_averages_for_plotting = {}
    overall_sem_for_plotting = {}
    # Initialize variables to store the global min and max values
    global_min_val = float('inf')  # Set to infinity initially
    global_max_val = float('-inf')  # Set to negative infinity initially
    
    # Generate labels and plot each condition
    for index, output_name in enumerate(output_names):
        # label = output_name.split("_")[1]  # OR extract label from output name instead of plotting parameters dict. Up to you.
        overall_averages_for_plotting[output_name] = overall_averages[roi][output_name]
        overall_sem_for_plotting[output_name] = overall_sems[roi][output_name]

        # Calculate the minimum value for this condition, including SEM
        current_min_val = min(overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name])
        # Calculate the maximum value for this condition, including SEM
        current_max_val = max(overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name])

        # Update the global min and max values if necessary
        global_min_val = min(global_min_val, current_min_val)
        global_max_val = max(global_max_val, current_max_val)

        # Optionally, add a small margin to the range
        margin = (global_max_val - global_min_val) * 0.05  # 5% of the range as margin
        global_min_val -= margin
        global_max_val += margin

        label = plotting_parameters[output_name]['condition_name'] # extract label from plotting parameters dict
        color = plotting_parameters[output_name]['color']
        line_style = plotting_parameters[output_name]['line_style']

        plt.plot(times, overall_averages_for_plotting[output_name], label=f'Average {roi} {label}', linestyle=line_style, color=color)
        plt.fill_between(times, overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name], overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name], alpha=0.3, color=color)

    plt.xlabel('Time (s)')
    plt.ylabel('Z-score')
    plt.title(f'Average {roi} Signal with Standard Error for {save_name}')
    plt.legend()
    # Adjust the y-axis limits
    plt.ylim([global_min_val, global_max_val])
    plt.savefig(save_path)
    plt.show()

this is just for congruent vs congruency proportion

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('dlpfc', save_dir, 'congruency_congruencyProportion', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('acc', save_dir, 'congruency_congruencyProportion', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('parietal', save_dir, 'congruency_congruencyProportion', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('dlpfc', save_dir, 'switchType_switchProportion', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('acc', save_dir, 'switchType_switchProportion', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('parietal', save_dir, 'switchType_switchProportion', overall_averages, overall_sems, output_names, plotting_parameters)

this just for congruency vs switch type

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('dlpfc', save_dir, 'congruency_switchType', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('acc', save_dir, 'congruency_switchType', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('parietal', save_dir, 'congruency_switchType', overall_averages, overall_sems, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('lpfc', save_dir, 'congruency_switchType', overall_averages, overall_sems, output_names, plotting_parameters)

try to plot different groups of electrodes based on significance for effects 4/4  

In [ ]:
def get_significant_electrodes_by_effect(effect_structure, effect_type):
    '''
    Extracts electrodes with significant specified effects.

    :param effect_structure: Dictionary containing significant effects for each electrode and subject.
    :param effect_type: Single string or list of strings specifying the effect(s) of interest ("congruency", "switchType", "congruency:switchType", etc).
    :return: A dictionary mapping subject IDs to lists of electrodes with the specified significant effect(s).
    '''
    significant_electrodes = {}

    # Ensure effect_type is a list to simplify the logic
    if isinstance(effect_type, str):
        effect_type = [effect_type]

    for subject_id, electrodes in effect_structure.items():
        significant_electrodes_for_subject = []  # Temporary list to hold significant electrodes for a subject
        for electrode, effects in electrodes.items():
            for effect, p_value in effects:  # Directly unpacking the tuples here
                if effect in effect_type and p_value < 0.05:  # Check if effect is among those of interest
                    significant_electrodes_for_subject.append(electrode)
                    break  # Stop checking once a significant effect is found for this electrode
        if significant_electrodes_for_subject:  # Only add to the dict if there are significant electrodes for the subject
            significant_electrodes[subject_id] = significant_electrodes_for_subject
    return significant_electrodes

In [ ]:
congruencySigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency')
switchTypeSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'switchType')
congruencySwitchTypeInteractionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency:switchType')
allEffectSensitiveElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, ['congruency', 'switchType', 'congruency:switchType'])

In [ ]:
allEffectSensitiveElectrodes

remake overall averages but just for the chosen electrodes. I wonder if I should make this a function...

In [ ]:
# Time windows
# define these based on epoch start and also sampling rate
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072 #define these based on the actual sampling rate variable in the future...i think i do this in another script somewhere.
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096
# Function to create a nested dictionary for each output_name with ROIs as keys
def initialize_output_data(rois, output_names):
    return {output_name: {roi: [] for roi in rois} for output_name in output_names}

def is_dict_of_dicts(d):
    """Check if the input is a dictionary of dictionaries."""
    return isinstance(d, dict) and all(isinstance(val, dict) for val in d.values())

def get_sig_electrodes(sig_electrodes, roi, sub):
    """Get significant electrodes based on the structure of sig_electrodes. Significant effects structure removes roi info, but sig_electrodes_per_subject_roi keeps it.
    maybe I should make these consistent... 4/5"""
    if is_dict_of_dicts(sig_electrodes):
        return sig_electrodes[roi].get(sub, [])
    else:
        return sig_electrodes.get(sub, [])
    
# this massive function needs to be split up and replace its above non-function form once it works 4/5.
def get_average_data_for_specific_electrodes(subjects, rois, output_names, sig_electrodes):
    # Assuming output_names contains all four conditions
    # Initializing dictionaries
    output_data_trialAvg_lists = initialize_output_data(rois, output_names)
    output_data_trialStd_lists = initialize_output_data(rois, output_names)
    output_data_timeAvg_firstHalfSecond_lists = initialize_output_data(rois, output_names)
    output_data_timeAvg_secondHalfSecond_lists = initialize_output_data(rois, output_names)
    output_data_timeAvg_fullSecond_lists = initialize_output_data(rois, output_names)

    # Initialize a dictionary to hold mappings
    overall_electrode_mapping = []

    # Initialize a dictionary to hold mappings for each ROI
    electrode_mapping_per_roi = {roi: [] for roi in rois}
    for sub in subjects:
        for roi in rois:
            for output_name in output_names:
                # Determine significant electrodes for the current ROI and subject. Hmm not sure if this will work if I have more than one ROI for the 
                # get_significant_electrodes_by_effect electrodes.. 4/5
                sig_electrodes_this_sub = get_sig_electrodes(sig_electrodes, roi, sub)
                print('sub', sub)
                print('sig elecs:', sig_electrodes_this_sub)
                
                if not sig_electrodes_this_sub:  # Skip if no significant electrodes
                    continue
                            
                for electrode in sig_electrodes_this_sub:
                    # For each significant electrode, append a tuple to the mapping list
                    # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
                    # The index can be the current length of the list before appending
                    index = len(overall_electrode_mapping)
                    overall_electrode_mapping.append((sub, roi, electrode, output_name, index))  

                    # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
                    # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
                    index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
                    electrode_mapping_per_roi[roi].append((sub, electrode, output_name, index))
                    
                # Load trial-level data for the current condition and pick significant electrodes
                epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes_this_sub)
                # print(epochs.get_data().shape)
                # Calculate averages for each time window
                trial_avg, trial_std, time_avg_firstHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
                _, _, time_avg_secondHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
                _, _, time_avg_fullSecond = utils.filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)
                print('time avg full second shape:', time_avg_fullSecond.shape)

                # Append the results to their respective lists
                output_data_trialAvg_lists[output_name][roi].append(trial_avg)
                output_data_trialStd_lists[output_name][roi].append(trial_std)
                output_data_timeAvg_firstHalfSecond_lists[output_name][roi].append(time_avg_firstHalfSecond)
                output_data_timeAvg_secondHalfSecond_lists[output_name][roi].append(time_avg_secondHalfSecond)
                output_data_timeAvg_fullSecond_lists[output_name][roi].append(time_avg_fullSecond)


    # After collecting all data, concatenate across subjects for each roi and condition
    concatenated_trialAvg_data = {}
    concatenated_trialStd_data = {}

    for roi in rois:
        concatenated_trialAvg_data[roi] = {}
        concatenated_trialStd_data[roi] = {}

        for output_name in output_names:
            concatenated_trialAvg_data[roi][output_name] = np.concatenate(output_data_trialAvg_lists[output_name][roi], axis=0)
            concatenated_trialStd_data[roi][output_name] = np.concatenate(output_data_trialStd_lists[output_name][roi], axis=0)


    # Calculate mean and SEM across electrodes for all time windows and rois
    overall_averages = {}
    overall_sems = {}
    for roi in rois:
        overall_averages[roi] = {}
        overall_sems[roi] = {}
        for output_name in output_names:
            trialAvg_data = concatenated_trialAvg_data[roi][output_name]
            overall_averages[roi][output_name] = np.nanmean(trialAvg_data, axis=0)
            overall_sems[roi][output_name] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])
    
    return concatenated_trialAvg_data, concatenated_trialStd_data, overall_averages, overall_sems, output_data_timeAvg_fullSecond_lists

In [ ]:
concatenated_trialAvg_data_congruencySigElectrodes, concatenated_trialStd_data_congruencySigElectrodes, congruencySigElectrodesAverage, congruencySigElectrodesSEM, congruencySigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, output_names, congruencySigElectrodes)
concatenated_trialAvg_data_switchTypeSigElectrodes, concatenated_trialStd_data_switchTypeSigElectrodes, switchTypeSigElectrodesAverage, switchTypeSigElectrodesSEM, switchTypeSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, output_names, switchTypeSigElectrodes)
concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes, concatenated_trialStd_data_congruencySwitchTypeInteractionSigElectrodes, congruencySwitchTypeInteractionSigElectrodesAverage, congruencySwitchTypeInteractionSigElectrodesSEM, congruencySwitchTypeInteractionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, output_names, congruencySwitchTypeInteractionSigElectrodes)
concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes, concatenated_trialStd_data_congruencySwitchTypeInteractionSigElectrodes, allEffectSensitiveElectrodesAverage, allEffectSensitiveElectrodesSEM, allEffectSensitiveElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, output_names, allEffectSensitiveElectrodes)

finally plot the average traces for each output name for these chosen electrodes

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('lpfc', save_dir, 'congruency_switchType_congruencySigElectrodes', congruencySigElectrodesAverage, congruencySigElectrodesSEM, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('lpfc', save_dir, 'congruency_switchType_switchTypeSigElectrodes', switchTypeSigElectrodesAverage, switchTypeSigElectrodesSEM, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('lpfc', save_dir, 'congruency_switchType_congruencySwitchTypeInteractionSigElectrodes', congruencySwitchTypeInteractionSigElectrodesAverage, switchTypeSigElectrodesSEM, output_names, plotting_parameters)

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_roi('lpfc', save_dir, 'congruency_switchType_allEffectSensitiveElectrodes', allEffectSensitiveElectrodesAverage, allEffectSensitiveElectrodesSEM, output_names, plotting_parameters)

plot (ir - cr) vs (is - cs)  
4/5 - functionize this stuff later

SEM diff = sqrt(SEM1^2 + SEM2^2)

In [ ]:
diff_ir_cr = {}  # Difference between IR and CR
diff_is_cs = {}  # Difference between IS and CS

for roi in rois:
    diff_ir_cr[roi] = congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'] - congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror']
    diff_is_cs[roi] = congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'] - congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror']

sem_diff_ir_cr = {}
sem_diff_is_cs = {}

for roi in rois:
    sem_diff_ir_cr[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2))
    sem_diff_is_cs[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2))

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, diff_ir_cr[roi], label='IR - CR', color='black', linestyle='-')
ax.plot(times, diff_is_cs[roi], label='IS - CS', color='black', linestyle='--')

ax.fill_between(times, diff_ir_cr[roi] - sem_diff_ir_cr[roi], diff_ir_cr[roi] + sem_diff_ir_cr[roi], alpha=0.2, color='black')
ax.fill_between(times, diff_is_cs[roi] - sem_diff_is_cs[roi], diff_is_cs[roi] + sem_diff_is_cs[roi], alpha=0.2, color='black')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

now let's do switch cost as a function of congruency

In [ ]:
congruencySwitchTypeInteractionSigElectrodesAverage

In [ ]:
diff_is_ir = {}  # Difference between IR and CR
diff_cs_cr = {}  # Difference between IS and CS

for roi in rois:
    diff_is_ir[roi] = congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'] - congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror']
    diff_cs_cr[roi] = congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'] - congruencySwitchTypeInteractionSigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror']

sem_diff_is_ir = {}
sem_diff_cs_cr = {}

for roi in rois:
    sem_diff_is_ir[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2))
    sem_diff_cs_cr[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2))

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, diff_is_ir[roi], label='IS - IR', color='black', linestyle='-')
ax.plot(times, diff_cs_cr[roi], label='CS - CR', color='black', linestyle='--')

ax.fill_between(times, diff_is_ir[roi] - sem_diff_is_ir[roi], diff_is_ir[roi] + sem_diff_is_ir[roi], alpha=0.2, color='black')
ax.fill_between(times, diff_cs_cr[roi] - sem_diff_cs_cr[roi], diff_cs_cr[roi] + sem_diff_cs_cr[roi], alpha=0.2, color='black')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

let's also plot i vs c 4/5

In [ ]:
avg_ir_is = {}  # Average of IR and IS
avg_cr_cs = {}  # Average of CR and CS

for roi in rois:
    avg_ir_is[roi] = (congruencySigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'] + congruencySigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
    avg_cr_cs[roi] = (congruencySigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'] + congruencySigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_is = {}
avg_sem_cr_cs = {}

for roi in rois:
    avg_sem_ir_is[roi] = np.sqrt((np.power(congruencySigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2)) / 2)
    avg_sem_cr_cs[roi] = np.sqrt((np.power(congruencySigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_cr_cs[roi], label='Congruent', color='red', linestyle='-')
ax.plot(times, avg_ir_is[roi], label='Incongruent', color='red', linestyle='--')

ax.fill_between(times, avg_ir_is[roi] - avg_sem_ir_is[roi], avg_ir_is[roi] + avg_sem_ir_is[roi], alpha=0.2, color='red')
ax.fill_between(times, avg_cr_cs[roi] - avg_sem_cr_cs[roi], avg_cr_cs[roi] + avg_sem_cr_cs[roi], alpha=0.2, color='red')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

now lets do i - c for the switch type main effect electrodes

In [ ]:
avg_ir_is = {}  # Average of IR and IS
avg_cr_cs = {}  # Average of CR and CS

for roi in rois:
    avg_ir_is[roi] = (switchTypeSigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'] + switchTypeSigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
    avg_cr_cs[roi] = (switchTypeSigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'] + switchTypeSigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_is = {}
avg_sem_cr_cs = {}

for roi in rois:
    avg_sem_ir_is[roi] = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2)) / 2)
    avg_sem_cr_cs[roi] = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_is[roi], label='Congruent', color='red', linestyle='-')
ax.plot(times, avg_cr_cs[roi], label='Incongruent', color='red', linestyle='--')

ax.fill_between(times, avg_ir_is[roi] - avg_sem_ir_is[roi], avg_ir_is[roi] + avg_sem_ir_is[roi], alpha=0.2, color='red')
ax.fill_between(times, avg_cr_cs[roi] - avg_sem_cr_cs[roi], avg_cr_cs[roi] + avg_sem_cr_cs[roi], alpha=0.2, color='red')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

now let's plot switch vs repeat for the switch type main effect electrodes

In [ ]:
avg_ir_cr = {}  # Average of IR and CR
avg_is_cs = {}  # Average of IS and CS

for roi in rois:
    avg_ir_cr[roi] = (switchTypeSigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'] + switchTypeSigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
    avg_is_cs[roi] = (switchTypeSigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'] + switchTypeSigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_cr = {}
avg_sem_is_cs = {}

for roi in rois:
    avg_sem_ir_cr[roi] = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
    avg_sem_is_cs[roi] = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_cr[roi], label='Repeat', color='blue', linestyle='-')
ax.plot(times, avg_is_cs[roi], label='Switch', color='blue', linestyle='--')

ax.fill_between(times, avg_ir_cr[roi] - avg_sem_ir_cr[roi], avg_ir_cr[roi] + avg_sem_ir_cr[roi], alpha=0.2, color='blue')
ax.fill_between(times, avg_is_cs[roi] - avg_sem_is_cs[roi], avg_is_cs[roi] + avg_sem_is_cs[roi], alpha=0.2, color='blue')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

interestingly, the switch vs repeat for congruency main effect electrodes is quite different too

In [ ]:
avg_ir_cr = {}  # Average of IR and CR
avg_is_cs = {}  # Average of IS and CS

for roi in rois:
    avg_ir_cr[roi] = (congruencySigElectrodesAverage[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'] + congruencySigElectrodesAverage[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
    avg_is_cs[roi] = (congruencySigElectrodesAverage[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'] + congruencySigElectrodesAverage[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_cr = {}
avg_sem_is_cs = {}

for roi in rois:
    avg_sem_ir_cr[roi] = np.sqrt((np.power(congruencySigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
    avg_sem_is_cs[roi] = np.sqrt((np.power(congruencySigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(congruencySigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_cr[roi], label='repeat', color='blue', linestyle='-')
ax.plot(times, avg_is_cs[roi], label='switch', color='blue', linestyle='--')

ax.fill_between(times, avg_ir_cr[roi] - avg_sem_ir_cr[roi], avg_ir_cr[roi] + avg_sem_ir_cr[roi], alpha=0.2, color='blue')
ax.fill_between(times, avg_is_cs[roi] - avg_sem_is_cs[roi], avg_is_cs[roi] + avg_sem_is_cs[roi], alpha=0.2, color='blue')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
plt.show()

### make bar plots windowed from 0 to 1 s of my two conditions 4/9  
greg wants to see these

first get mean and sem across electrodes from 0 to 1 for various conditions

In [ ]:
def calculate_means_sems_from_fullSecond_averages(timeAvg_fullSecond_lists, rois, output_names):
    means = {}
    sems = {}
    
    for roi in rois:
        means[roi] = {}
        sems[roi] = {}
        for output_name in output_names:
            # Handle each electrode's full-second average individually
            full_second_averages = timeAvg_fullSecond_lists[output_name][roi]
            # Assuming full_second_averages is a list of numpy arrays, one per electrode
            
            # Initialize lists to store the mean for each electrode
            electrode_means = []
            
            for electrode_data in full_second_averages:
                # Calculate the mean for this electrode in the full second window
                electrode_mean = np.nanmean(electrode_data)
                electrode_means.append(electrode_mean)
            
            # Convert the list of means to a NumPy array for further calculation
            electrode_means = np.array(electrode_means)
            
            # Calculate the overall mean and SEM across electrodes
            condition_mean = np.nanmean(electrode_means)
            condition_sem = np.std(electrode_means, ddof=1) / np.sqrt(len(electrode_means))
            
            means[roi][output_name] = condition_mean
            sems[roi][output_name] = condition_sem
    
    return means, sems

# Now, calculate the means and SEMs
means_windowed_full_second, sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(congruencySwitchTypeInteractionSigElectrodes_timeAvg_fullSecondLists, rois, output_names)
congruency_means_windowed_full_second, congruency_sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(congruencySigElectrodes_timeAvg_fullSecondLists, rois, output_names)
switchType_means_windowed_full_second, switchType_sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(switchTypeSigElectrodes_timeAvg_fullSecondLists, rois, output_names)

now plot all four conditions as bars on one plot for the interaction effect 4/10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Corrected condition names based on your dataset
conditions = [
    'Stimulus_cr_fixationCrossBase_1sec_mirror',  # Congruent Repeat
    'Stimulus_ir_fixationCrossBase_1sec_mirror',  # Incongruent Repeat
    'Stimulus_cs_fixationCrossBase_1sec_mirror',  # Congruent Switch
    'Stimulus_is_fixationCrossBase_1sec_mirror'   # Incongruent Switch
]

# Retrieve the means and SEMs for each condition for plotting
means = [means_windowed_full_second[roi][cond] for cond in conditions]
sems = [sems_windowed_full_second[roi][cond] for cond in conditions]

# Colors for each condition for plotting
group_colors = ['pink', 'red', 'pink', 'red']  # Mapping colors to Congruent/Incongruent

# Plotting
fig, ax = plt.subplots()
bar_width = 0.35  # Width of the bars
index = np.arange(2)  # Two groups: Repeat and Switch

# Creating bars for each group
for i, (mean, sem, color) in enumerate(zip(means, sems, group_colors)):
    position = index[i // 2] + (i % 2 - 0.5) * bar_width
    ax.bar(position, mean, yerr=sem, capsize=5, color=color, width=bar_width)

# Ensure no labels or ticks are shown on the x-axis
ax.set_xticks([])  # No x-tick marks
ax.tick_params(axis='x', which='both', length=0)  # No x-tick marks
ax.tick_params(axis='y', labelsize=32)  # Adjust y-tick label size as needed

# Customizing the plot (commented sections are optional customizations)
# ax.set_ylabel('Average Z-score', fontsize=14)
# ax.set_title('Average Z-score From Baseline by Condition and Type (Full Second)', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()


now plot congruent vs incongruent 4/10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming roi, congruency_means_windowed_full_second, and congruency_sems_windowed_full_second are defined

# Conditions based on your specification
conditions = [
    'Stimulus_cr_fixationCrossBase_1sec_mirror',  # Congruent Repeat
    'Stimulus_ir_fixationCrossBase_1sec_mirror',  # Incongruent Repeat
    'Stimulus_cs_fixationCrossBase_1sec_mirror',  # Congruent Switch
    'Stimulus_is_fixationCrossBase_1sec_mirror'   # Incongruent Switch
]

# Calculating combined means and SEMs for Congruent and Incongruent conditions
combined_means = [
    np.mean([
        congruency_means_windowed_full_second[roi][conditions[0]],  # CR mean
        congruency_means_windowed_full_second[roi][conditions[2]]   # CS mean
    ]),
    np.mean([
        congruency_means_windowed_full_second[roi][conditions[1]],  # IR mean
        congruency_means_windowed_full_second[roi][conditions[3]]   # IS mean
    ])
]

combined_sems = [
    np.sqrt(np.mean([
        congruency_sems_windowed_full_second[roi][conditions[0]]**2,  # CR SEM^2
        congruency_sems_windowed_full_second[roi][conditions[2]]**2   # CS SEM^2
    ])),
    np.sqrt(np.mean([
        congruency_sems_windowed_full_second[roi][conditions[1]]**2,  # IR SEM^2
        congruency_sems_windowed_full_second[roi][conditions[3]]**2   # IS SEM^2
    ]))
]

# Plotting adjustments for the first plot
fig, ax = plt.subplots()
bar_width = 0.015  # Narrower bars

# Adjust the index to create slight separation
index = np.array([0.4375, 0.4625])  # Adjusted for slight separation

bars = ax.bar(index, combined_means, yerr=combined_sems, capsize=5, color=['pink', 'red'], width=bar_width)

# Customizing the plot
ax.set_xticks(index)
ax.set_xticklabels(['Congruent', 'Incongruent'], fontsize=24)
ax.tick_params(axis='y', labelsize=24)

plt.tight_layout()
plt.show()

# Plotting adjustments for the second plot with same formatting
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed

# Reusing the same bar width and indices for consistency
bars = ax.bar(index, combined_means, 
              yerr=combined_sems, capsize=5, 
              color=['pink', 'red'], width=bar_width)

ax.set_xticks(index)
ax.set_xticklabels(['Congruent', 'Incongruent'], fontsize=32)
ax.tick_params(axis='y', labelsize=32)

# Optional: Uncomment the next line if you want to see the effect without plt.tight_layout()
# plt.tight_layout()

plt.show()


now plot switch vs repeat windowed from 0 to 1

In [ ]:
combined_means_switch_repeat = [
    np.mean([
        switchType_means_windowed_full_second[roi][conditions[0]],  # CR mean
        switchType_means_windowed_full_second[roi][conditions[1]]   # IR mean
    ]),
    np.mean([
        switchType_means_windowed_full_second[roi][conditions[2]],  # CS mean
        switchType_means_windowed_full_second[roi][conditions[3]]   # IS mean
    ])
]

combined_sems_switch_repeat = [
    np.sqrt(np.mean([
        switchType_sems_windowed_full_second[roi][conditions[0]]**2,  # CR SEM^2
        switchType_sems_windowed_full_second[roi][conditions[1]]**2   # IR SEM^2
    ])),
    np.sqrt(np.mean([
        switchType_sems_windowed_full_second[roi][conditions[2]]**2,  # CS SEM^2
        switchType_sems_windowed_full_second[roi][conditions[3]]**2   # IS SEM^2
    ]))
]

fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed
bar_width = 0.015  # Even narrower bars

# Closer indices, but ensure they're distinct enough to not overlap
index_switch_repeat = np.array([0.4375, 0.4625])

bars = ax.bar(index_switch_repeat, combined_means_switch_repeat, 
              yerr=combined_sems_switch_repeat, capsize=5, 
              color=['lightblue', 'blue'], width=bar_width)

ax.set_xticks(index_switch_repeat)
ax.set_xticklabels(['Repeat', 'Switch'], fontsize=24)
ax.tick_params(axis='y', labelsize=24)

# Uncomment the next line if you want to see the effect without plt.tight_layout()
# plt.tight_layout()

plt.show()


### plot individual electrodes for interaction effects
i think this will just work regardless of the output names 3/5

test this new plot significance function that offsets for each significance bar 4/7

In [ ]:
# def plot_significance(ax, times, sig_effects, y_offset=0.1):
#     """
#     Plot significance bars for the effects on top of the existing axes, adjusted for time windows.

#     Parameters:
#     - ax: The matplotlib Axes object to plot on.
#     - times: Array of time points for the x-axis.
#     - sig_effects: Dictionary with time windows as keys and lists of tuples (effect, p-value) as values.
#     - y_offset: The vertical offset between different time window significance bars.
#     """
#     y_pos_base = ax.get_ylim()[1]  # Get the top y-axis limit to place significance bars

#     time_windows = {
#         'FirstHalfSecond': (0, 0.5),
#         'SecondHalfSecond': (0.5, 1),
#         'FullSecond': (0, 1)
#     }

#     window_offsets = {window: 0 for window in time_windows}  # Initialize offsets for each time window

#     # Sort time windows to ensure 'FullSecond' bars are plotted last (on top)
#     for time_window, effects in sorted(sig_effects.items(), key=lambda x: x[0] == 'FullSecond'):
#         base_y_pos = y_pos_base + y_offset * list(time_windows).index(time_window)
#         for effect, p_value in effects:
#             start_time, end_time = time_windows[time_window]
#             # Adjust y_pos based on how many bars have already been plotted in this window
#             y_pos = base_y_pos + y_offset * window_offsets[time_window]

#             # Update the color selection logic as per your requirement
#             color = 'black'  # Default color for unmatched conditions
                        
#             if 'congruency' in effect:
#                 color = 'red'
#             elif 'congruencyProportion' in effect:
#                 color = 'green'
#             elif 'switchType' in effect:
#                 color = 'blue'
#             elif 'switchProportion' in effect:
#                 color = 'yellow'
#             elif 'congruency:congruencyProportion' in effect:
#                 color = 'purple'
#             elif 'switchType:switchProportion' in effect:
#                 color = 'yellowgreen'
#             elif 'congruency:switchType' in effect:
#                 color = 'black'

#             num_asterisks = '*' * (1 if p_value < 0.05 else 2 if p_value < 0.01 else 3)
#             ax.plot([start_time, end_time], [y_pos, y_pos], color=color, lw=4)
#             ax.text((start_time + end_time) / 2, y_pos, num_asterisks, ha='center', va='bottom', color=color)

#             window_offsets[time_window] += 1  # Increment the offset for this time window

In [ ]:
def plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1):
    """
    Plot significance bars for the effects on top of the existing axes, adjusted for time windows.

    Parameters:
    - ax: The matplotlib Axes object to plot on.
    - times: Array of time points for the x-axis.
    - sig_effects: Dictionary with time windows as keys and lists of tuples (effect, p-value) as values.
    - y_offset: The vertical offset between different time window significance bars.
    """
    y_pos_base = ax.get_ylim()[1]  # Get the top y-axis limit to place significance bars

    time_windows = {
        'FirstHalfSecond': (0, 0.5),
        'SecondHalfSecond': (0.5, 1),
        'FullSecond': (0, 1)
    }

    window_offsets = {window: 0 for window in time_windows}  # Initialize offsets for each time window

    # Sort time windows to ensure 'FullSecond' bars are plotted last (on top)
    for time_window, effects in sorted(sig_effects.items(), key=lambda x: x[0] == 'FullSecond'):
        base_y_pos = y_pos_base + y_offset * list(time_windows).index(time_window)
        y_pos = base_y_pos
        for effect, p_value in effects:
            if 'congruency:switchType' in effect:
                start_time, end_time = time_windows[time_window]
                # Adjust y_pos based on how many bars have already been plotted in this window
                y_pos = base_y_pos + y_offset * window_offsets[time_window]

                # Update the color selection logic as per your requirement
                color = 'black'  # Default color for unmatched conditions

                num_asterisks = '*' * (1 if p_value < 0.05 else 2 if p_value < 0.01 else 3)
                ax.plot([start_time, end_time], [y_pos, y_pos], color=color, lw=4)
                ax.text((start_time + end_time) / 2, y_pos, num_asterisks, ha='center', va='bottom', color=color)

                window_offsets[time_window] += 1  # Increment the offset for this time window
            else:
                continue


In [ ]:
congruencySwitchTypeInteractionSigElectrodes

In [ ]:
LAB_root = None
channels = None

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

# # Use the times from your evoked data (assuming these are representative for all subjects)
# times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

# port over the plot_electrodes_grid_whole_brain_analysis here, but replace wherever the save name is wholebrainanalysis with the roi names. 3/25.
def plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        for output_name in output_names:
            color = plotting_parameters[output_name]['color']
            line_style = plotting_parameters[output_name]['line_style']
            ax.plot(times, data[output_name], label=f'{roi}_{output_name}', color=color, linestyle=line_style)
            ax.fill_between(times, 
                            data[output_name] - np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])),
                            data[output_name] + np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])), alpha=0.3)

        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

        # Retrieve significant effects for the current subject and electrode
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            # Adjust y_offset based on plotting needs. This used to not be assigned to a variable. 3/20.
            utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()


test looping over subjects and electrodes as a function 4/1

In [ ]:
def plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, roi, concatenated_trialAvg_data, output_names, grid_size, save_dir, save_name, times, plotting_parameters):
    electrodes_data = []
    electrode_counter = 0
    grid_num = 0

    # Load in significant effects structure
    significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
    with open(significant_effects_structure_file_path, 'r') as file:
        significant_effects_structure = json.load(file)

    for sub in subjects:
        if sub in sig_electrodes_per_subject_roi[roi]:
            for electrode in sig_electrodes_per_subject_roi[roi][sub]:
                electrode_data = {}
                for output_name in output_names:
                    # Ensure the index is correctly used here for your data structure
                    electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

                electrodes_data.append((electrode_data, sub, electrode))
                electrode_counter += 1
                if len(electrodes_data) == grid_size:
                    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                    electrodes_data = []  # Reset for the next grid
                    grid_num += 1

    # Plot remaining electrodes in the last grid
    if electrodes_data:
        plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)


dlpfc

In [ ]:
plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, 'dlpfc', concatenated_trialAvg_data, output_names, 16, save_dir, save_name, times, plotting_parameters)

acc

In [ ]:
plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, 'acc', concatenated_trialAvg_data, output_names, 16, save_dir, save_name, times, plotting_parameters)

parietal

In [ ]:
plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, 'parietal', concatenated_trialAvg_data, output_names, 16, save_dir, save_name, times, plotting_parameters)

lpfc

In [ ]:
plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, 'lpfc', concatenated_trialAvg_data, output_names, 16, save_dir, save_name, times, plotting_parameters)

awful godforsaken code to get individual electrode plots for congruency main effects, switch main effects, and interaction effects for lpfc 4/8  
replace with function later after CNS...    

In [ ]:
LAB_root = None
channels = None

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json

def plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate S and R for the electrode
        avg_ir_cr = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
        avg_is_cs = (data['Stimulus_is_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
        # avg_sem_is_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
        # avg_sem_cr_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

        # Plotting S and R
        ax.plot(times, avg_ir_cr, label='repeat', color='blue', linestyle='-')
        ax.plot(times, avg_is_cs, label='switch', color='blue', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()


def plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate I-C for the electrode
        avg_ir_is = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
        avg_cr_cs = (data['Stimulus_cr_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
        # avg_sem_is_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
        # avg_sem_cr_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

        # Plotting I-C difference
        ax.plot(times, avg_cr_cs, label='congruent', color='red', linestyle='-')
        ax.plot(times, avg_ir_is, label='incongruent', color='red', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



def plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate congruency effect as a function of switch type for the electrode

        avg_diff_ir_cr = data['Stimulus_ir_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']
        avg_diff_is_cs = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_cs_fixationCrossBase_1sec_mirror']


        ax.plot(times, avg_diff_ir_cr, label='IR - CR', color='black', linestyle='-')
        ax.plot(times, avg_diff_is_cs, label='IS - CS', color='black', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1) #change back to plot_significance after poster 4/8

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



def plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate congruency effect as a function of switch type for the electrode

        avg_diff_is_ir = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_ir_fixationCrossBase_1sec_mirror']
        avg_diff_cs_cr = data['Stimulus_cs_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']


        ax.plot(times, avg_diff_is_ir, label='IS - IR', color='black', linestyle='-')
        ax.plot(times, avg_diff_cs_cr, label='CS - CR', color='black', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1) #change back to plot_significance after poster 4/8

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



try this for lpfc 4/8  
just for poster, clean this up!! this is hella hard-coded 

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'congruencySigElectrodesCongruencyComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access congruencySigElectrodes for sub
    # If sub is not a key, congruencySigElectrodes_for_sub will be None
    congruencySigElectrodes_for_sub = congruencySigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySigElectrodes_for_sub:
        electrode_data = {}
        for output_name in output_names:
            electrode_data[output_name] = concatenated_trialAvg_data_congruencySigElectrodes[roi][output_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

okay now do individual for switch type

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'switchTypeSigElectrodesSwitchTypeComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    switchTypeSigElectrodes_for_sub = switchTypeSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if switchTypeSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in switchTypeSigElectrodes_for_sub:
        electrode_data = {}
        for output_name in output_names:
            electrode_data[output_name] = concatenated_trialAvg_data_switchTypeSigElectrodes[roi][output_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

now do individual for interaction (congruency effect by switch type)

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySwitchTypeInteractionSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySwitchTypeInteractionSigElectrodes_for_sub:
        electrode_data = {}
        for output_name in output_names:
            electrode_data[output_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][output_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

now do individual for interaction (switch cost by congruency)

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'switchCostCongruencyInteractionSigElectrodes'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySwitchTypeInteractionSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySwitchTypeInteractionSigElectrodes_for_sub:
        electrode_data = {}
        for output_name in output_names:
            electrode_data[output_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][output_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

okay once we choose the electrodes we want, plot the single example electrode

for congruency

In [ ]:
def plot_single_electrode_data_congruency(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size
    avg_ir_is = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
    avg_cr_cs = (data['Stimulus_cr_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
    
    ax.plot(times, avg_cr_cs, label='Congruent', color='red', linestyle='-')
    ax.plot(times, avg_ir_is, label='Incongruent', color='red', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0063', 'RMMF13'
roi = 'lpfc'
electrode_index = 5 # right now just manually count this from the grid plot BUT make this real after CNS 4/9

save_name = 'congruencySigElectrodesCongruencyComparison'
congruencySigElectrodes_for_sub = congruencySigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data_congruencySigElectrodes[roi][output_name][electrode_index]

plot_single_electrode_data_congruency(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

for switchType

In [ ]:
def plot_single_electrode_data_switchType(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size
    avg_ir_cr = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
    avg_is_cs = (data['Stimulus_is_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
    
    ax.plot(times, avg_ir_cr, label='Repeat', color='blue', linestyle='-')
    ax.plot(times, avg_is_cs, label='Switch', color='blue', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        plot_significance(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0059', 'LMMF9'
roi = 'lpfc'
electrode_index = 1 # right now just manually get this from the grid plot BUT make this real after CNS 4/9

save_name = 'switchTypeSigElectrodesCongruencyComparison'
switchTypeSigElectrodes_for_sub = switchTypeSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data_switchTypeSigElectrodes[roi][output_name][electrode_index]

plot_single_electrode_data_switchType(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

for interaction effect, congruency effect by switch type

In [ ]:
def plot_single_electrode_data_interaction_effect(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_diff_ir_cr = data['Stimulus_ir_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']
    avg_diff_is_cs = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_cs_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_diff_ir_cr, label='IR - CR', color='black', linestyle='-')
    ax.plot(times, avg_diff_is_cs, label='IS - CS', color='black', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)
    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LFAI9'
electrode_index = 8 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'
congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][output_name][electrode_index]
plot_single_electrode_data_interaction_effect(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

In [ ]:
congruencySwitchTypeInteractionSigElectrodes

interaction effect for switch cost by congruency

In [ ]:
def plot_single_electrode_data_interaction_effect(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_diff_is_ir = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_ir_fixationCrossBase_1sec_mirror']
    avg_diff_cs_cr = data['Stimulus_cs_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_diff_is_ir, label='IS - IR', color='black', linestyle='-')
    ax.plot(times, avg_diff_cs_cr, label='CS - CR', color='black', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)
    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    ax.set_ylabel('Z-score Difference', fontsize=20)
    ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LFAI9'
electrode_index = 8 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'
congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][output_name][electrode_index]
plot_single_electrode_data_interaction_effect(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

plot example electrode with all four conditions  
note this index is in the overall sig electrodes

In [ ]:
def plot_single_electrode_data_interaction_effect_all_four_conditions(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_is = data['Stimulus_is_fixationCrossBase_1sec_mirror']
    avg_ir = data['Stimulus_ir_fixationCrossBase_1sec_mirror']
    avg_cs = data['Stimulus_cs_fixationCrossBase_1sec_mirror']
    avg_cr = data['Stimulus_cr_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_is, label='IS', color='red', linestyle='--')
    ax.plot(times, avg_ir, label='IR', color='red', linestyle='-')
    ax.plot(times, avg_cs, label='CS', color='pink', linestyle='--')
    ax.plot(times, avg_cr, label='CR', color='pink', linestyle='-')

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LPAI9'
electrode_index = 31 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'sigElectrodesPerSubjectROI_D0094LPAI9'
sig_electrodes_for_sub = sig_electrodes_per_subject_roi.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_index]
plot_single_electrode_data_interaction_effect_all_four_conditions(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

In [ ]:
# Example usage setup
sub, example_elec = 'D0065', 'RASF14'
electrode_index = 14 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'sigElectrodesPerSubjectROI_D0065_RASF14'
sig_electrodes_for_sub = sig_electrodes_per_subject_roi.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for output_name in output_names:
    electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_index]
plot_single_electrode_data_interaction_effect_all_four_conditions(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

old way of looping without a function 4/1

dlpfc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'dlpfc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

acc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'acc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

parietal

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'parietal'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

lpfc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)